In [128]:
import pandas as pd
import sqlalchemy
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import math

In [74]:
dbEngine = sqlalchemy.create_engine('sqlite:///../../database.db')

In [75]:
buyer_df = pd.read_sql(f"""
SELECT
  buyer, count(*)
FROM
  trades
WHERE 
  order_state = "consummated"
  AND order_date >= datetime('now', '-1 month')
GROUP BY
  buyer
""", dbEngine)

In [133]:
def get_df(buyer):
  return pd.read_sql(f"""
    SELECT
      order_time, product, price, price_unit, lot_unit
    FROM
      trades
    WHERE 
      order_state = "consummated"
      AND buyer = '{buyer}'
      AND order_time >= datetime('now', '-3 month')
    """, dbEngine, parse_dates=['order_time'])

def get_chart(buyer):
  df = get_df(buyer)
  nr = math.ceil(len(df.groupby("product"))/3)
  fig, axes = plt.subplots(nrows=nr, ncols=3, sharex=True, sharey=False, figsize=(14, nr*2.2))
  al = [i for j in axes for i in j] if nr > 1 else list(axes)
  for group, row in df.groupby("product"):
    ax = al.pop(0)
    row.plot(x="order_time", y="price", ax=ax, label=group, legend=False)
    ax.set_title(group)

  plt.tight_layout()
  for ax in al:
    ax.remove()
  return fig

In [132]:
buyer = widgets.Dropdown(
  options=buyer_df['buyer'].to_list(),
  description='Buyer:',
)
out = widgets.Output()
vbox = widgets.VBox(children=(buyer, out))
def handle_buyer(change):
  with out:
    clear_output(wait=True)
    display(get_chart(change['new']))
    
buyer.observe(handle_buyer, names='value')
display(vbox)

In [126]:
get_df("Aramco Trading Singapore Pte Ltd").groupby('product').count()

,order_time,price,price_unit,lot_unit
product,,,,
Platts GO,42,42,42,42
Platts GO (balmo),3,3,3,3
Platts GO 10ppm - AG,1,1,1,1
Platts GO Spr,2,2,2,2
Platts HSFO 380cst,12,12,12,12
Platts Jet,16,16,16,16
Platts Jet (balmo),10,10,10,10
Platts Jet Spr,13,13,13,13
Platts MTBE 3kt,4,4,4,4
